# Notebook experiment for model training (finally)

In [1]:
%pwd

'/Users/soogeunpark/Documents/text_summarizer_cicd/Text-Summarizer/research'

In [2]:
import os
os.chdir("..")

In [3]:
os.getcwd()

'/Users/soogeunpark/Documents/text_summarizer_cicd/Text-Summarizer'

First I need to update the ```config.yaml``` and ```params.yaml``` files.

Now will set up the ```entity```.

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    """
    Providing the format of the model trainer config
    """
    # first format for the parameters in the config.yaml
    root_dir: Path
    data_path: Path
    model_ckpt: Path # model_ckpt is also a path: we get it from google
    
    # now the parameters in param.yaml
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

Now writing the ```config/configuration.py: configuration manager```

In [5]:

from TextSummarizer.constants import *
# the asterisk imports everything in the directory
# the 'constants' includes the CONFIG_FILE_PATH and PARAMS_FILE_PATH

from TextSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        """
        This class is used to manage the configuration of the project
        """

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots]) 
        # this refers to the artifacts_roots in the config.yaml
        # this creates the 'artifacts' directory
        # because self.config does the "read_yaml" function which uses ConfigBox, the artifacts_roots can be just accessed by using the dot notation

    def model_trainer_config(self) -> ModelTrainerConfig:
        """
        This function returns the model trainer config
        """
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        
        return model_trainer_config


Now working on the components: this is the step that we are really defining the actions conducted by the ```ModelTrainer``` stage:

First, import libraries needed for training the model:

In [6]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


/Users/soogeunpark/Documents/text_summarizer_cicd/textsummarizer-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-01-03 11:55:08,405: INFO: config: PyTorch version 2.1.2 available.]


In [14]:
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        """
        This class is used to execute data validation
        """
        self.config = config
        
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # cuda is applicable only if the GPU is available
        # for Mac, cuda is not available. Instead, MPS can be utilized:
        
        device = "mps" if torch.backends.mps.is_available() else "cpu"
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_used = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_used)
        
        logger.info(f"model and tokenizer initiated: {self.config.model_ckpt}")
        
        # loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        logger.info("data loaded from disk")
        
        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size, # eval_batch_size same as train_batch_size
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.eval_steps, # save_steps used for eval_steps also
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )
        
        # training execution:
        trainer = Trainer(model=model_used, 
                          args=trainer_args, 
                          data_collator=seq2seq_data_collator, 
                          train_dataset=dataset_samsum_pt["test"], # USING TEST DATA, BECAUSE IT TAKES SHORTER TIME FOR NOW 
                          eval_dataset=dataset_samsum_pt["validation"])
        
        logger.info("training initiated")
        
        trainer.train()
        
        # saving model and tokenizer
        model_used.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_model_trained"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus_samsum_tokenizer"))
        
        logger.info("model and tokenizer saved")

Now creating the pipeline

In [15]:
try:
    config = ConfigurationManager()

    model_trainer_config = config.model_trainer_config()

    model_trainer = ModelTrainer(config = model_trainer_config)
    
    model_trainer.train()
    
except Exception as e:
    logger.exception(e)
    raise e
            

[2024-01-03 12:02:16,655: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-03 12:02:16,656: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-03 12:02:16,657: INFO: common: created directory at: artifacts]
[2024-01-03 12:02:16,657: INFO: common: created directory at: artifacts/model_trainer]
[2024-01-03 12:02:18,635: INFO: 1435184037: model and tokenizer initiated: mrm8488/t5-base-finetuned-summarize-news]
[2024-01-03 12:02:18,640: INFO: 1435184037: data loaded from disk]
[2024-01-03 12:02:18,646: INFO: 1435184037: training initiated]


  0%|          | 0/51 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[2024-01-03 12:02:19,293: ERROR: 4130022911: MPS backend out of memory (MPS allocated: 6.16 GB, other allocations: 14.27 GB, max allowed: 20.40 GB). Tried to allocate 5.88 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).]
Traceback (most recent call last):
  File "/var/folders/44/7h872h3167v5w7vlgzx7jfd80000gn/T/ipykernel_3495/4130022911.py", line 8, in <module>
    model_trainer.train()
  File "/var/folders/44/7h872h3167v5w7vlgzx7jfd80000gn/T/ipykernel_3495/1435184037.py", line 53, in train
    trainer.train()
  File "/Users/soogeunpark/Documents/text_summarizer_cicd/textsummarizer-venv/lib/python3.11/site-packages/transformers/trainer.py", line 1537, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Users/soogeunpark/Documents/text_summarizer_cicd/textsummarizer-venv/lib/python3.11/site-packages/transformers/trainer.py", line 1854, in _inner_training_loop
    tr_loss_

RuntimeError: MPS backend out of memory (MPS allocated: 6.16 GB, other allocations: 14.27 GB, max allowed: 20.40 GB). Tried to allocate 5.88 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).